# Computational Inquiry: How to Trust a Machine

In CMSE 202 (and 201 before it), you've been building a foundation of experience with fundamental tools and techniques essential to computational scientists.  Even if your interests don't end up taking you to an explicitly computational field of work or study, we still live in a computational world.  Whatever your path, the experience you're gaining better equips you to succeed in such a world.

This notebook introduces an activity that adds another brick to the foundation you're building.  You'll likely find it to be a bit different from your other CMSE classroom experiences.  That's because you'll be engaging in an _inquiry-based activity_.  The goal is to learn through inquiry and investigation, similar to the way science is done.  While it may feel a bit different, some of the scholars and scientists that study how people learn find it to be powerfully effective.  You'll soon be able to judge for yourself, but we hope you'll approach this activity with an open mind and a willingness to try something a bit different.

Alright, alright, so what are we actually trying to learn here? What tools and techniques will you get to play with?  I'm glad you asked.

This notebook focuses on introducing two concepts: **compartmental models** (which are a framework for describing a set of **ordinary differential equations (ODEs)**) and **finite differences**.  Now, you could easily spend up to an entire course on these.  The point here is to give the class just enough of a shared understanding of these concepts such that you can deploy them in your inquiry.  Some of you may already have some experience with these or related concepts, in which case this notebook can refresh and reinforce.  In either case, the plan for this activity is not to go into all the details and nuances, but to learn a bit and get some practical experience.  If your calculus is rusty or non-existent, there's no need to worry.  While we'll be playing with calculus concepts like differential equations and derivatives, neither past knowledge of them nor a calculus background is required for this activity.

That's because the focus of this activity is to develop practical, technical experience with the computational treatment of these concepts.  Even more, you're going to wrestle with the fundamental question of **how can we trust a machine?**.  What are some of the steps we can take, some of the analysis we can do, to convince ourselves and others that the results of a program running on a complex machine are meaningful?  In our everyday use of computers, they tend to act as mysterious "black boxes."  They do all sorts of things for us and we trust them to even if we don't understand the details, the same way we may trust and use a vehicle even if we don't understand all the details of how it works.  As a student of computational methods, however, you'll be writing your own computer code and using that of others.  In this activity, you'll get to look under the hood a bit and learn some of the ways we can evaluate the quality of a code.

#  Compartmental models, an ODE framework

In this section, we introduce the notion of a *compartmental model*.  While this modeling framework is often seen in the medical field (e.g. epidemiology, physiology, related medical imaging), it can be used as a general framework for building basic models of many systems of interest in a variety of scientific disciplines.  Compartmental models give us a way of describing how a system evolves over time, leading to something called an **ordinary differential equation (or ODE)**.  These look something like

$
\begin{equation}
    \frac{df}{dx} = g(x).
\end{equation}
$  

In a calculus course, you learn the formal mathematical definition of equations like this.  Informally, for the purposes of this activity, this differential equation is a way of describing the slope of some function $f(x)$ for any valid value of the independent variable $x$.  The right-hand-side, $g(x)$, gives us this slope.  So $g(0)$ is the slope at $x=0$.  Sometimes we describe a line's slope as "rise over run," or in fraction form: $\frac{\mathrm{rise}}{\mathrm{run}}$.  This is, roughly, what $\frac{df}{dx}$ is: it tells you the "rise" ($df$, change in $f(x)$) over the "run" ($dx$, change in $x$).  Lines like $f(x) = ax + b$ have the same slope (in this case, $a$) for _all_ $x$.  Generally, for some unknown, well-behaved function $f(x)$, the slope may be different at different values of $x$.  We'll often call a function "well-behaved" or "differentiable" if a well-defined slope exists for all $x$.  This is a detail we won't worry about too much here.  

A differential equation is said to be an _ordinary_ differential equation if it describes functions of _one_ independent variable (like $f(x)$).  Differential equations of functions that depend on multiple independent variables, such as functions of three spatial coordinates like $f(x, y, z)$, are _partial_ differential equations.  We won't worry about those in this activity.

The slope of a line is something you've had experience with and have some intuition for, so it's useful to introduce a differential equation this way.  But in the sciences and engineering, it can be useful to understand a differential equation as describing change, describing evolution with respect to some independent variable (often, but not always, either a spatial variable describing lengths like $x$ or a variable describing time like $t$).  These equations give us a powerful, mathematically robust way of modeling a system's behavior over time.  If we can develop a set of equations that describe this change, we can then use them to make predictions about a system's behavior, about what it will look like tomorrow based on what it looks like today.  In this activity, you will see concrete examples of this.

To help make things a bit more manageable and concrete, instead of drawing from the massive wealth of ODE-based models out there, we restrict ourselves to compartmental models.  A compartmental model consists of some number $N$ of compartments and a set of coefficients that describe how the compartments interact.  The evolution of each compartment $C_j$ in an $N$-compartment model is expressed as

$
\begin{equation}
    \frac{dC_j(t)}{dt} = f_j(u_j(t), C_1(t), ... , C_N(t)),
\end{equation}
$  
where $u_j(t)$ is an external function.  There may be one for each compartment ($u_j(t)$ for $j=1,2,...,N$), for only some compartments, or there may only be one specified for the whole model ($u(t)$).  Note that $j$ is an index variable that enumerates the compartments, going from 1 to $N$.

First, let's consider a 1-compartment model composed of an external funtion and a single compartment $C_1$:

$
\begin{equation}
    \frac{dC_1(t)}{dt} = K_1 u(t) - k_2 C_1(t).
\end{equation}
$

Loosely, this can be read as

$
\begin{equation}
    (\text{change in "amount" of }C_1\text{ over time})  =  \\
        (\text{rate of flow from } u \text{ to } C_1) (\text{"amount" in }u(t)) -
        (\text{rate of flow from } C_1 \text{ out of the system}) (\text{"amount" of }C_1).
\end{equation}
$

A diagram of this model can be given as:

![basic 1-compartment](figs/1comp_model_cartoon.png "Basic 1-compartment")

Though some real systems are modeled with a 1-compartment model, we can learn a bit more about using compartmental models by adding another compartment that can be interacted with.  You may be able to determine how we'd express such a system by analogy with the 1-compartment model.

We now have two compartments, so we'll want a $C_2(t)$. The quantity being tracked by a particular model can now flow in from the external function into $C_1$, which can in turn exchange with $C_2$, so we'll need some new coefficients for the rates describing such flows.  This could look like

$
\begin{equation}
    \frac{dC_1(t)}{dt} = K_1 u(t) - (k_2 + k_3) C_1(t) + k_4 C_2(t),\\
    \frac{dC_2(t)}{dt} = k_3 C_1(t) - k_4 C_2(t) - k_5 C_2(t).
\end{equation}
$

We can represent this with a diagram as

![basic 2-compartment](figs/2comp_model_cartoon.png "Basic 2-compartment")

You may realize this isn't the only way we could write a 2-compartment model.  Any given compartment may have (1) input from or output to an external function (like $u(t)$ into $C_1$ proportional to $K_1$), (2) flow out into another compartment (like from $C_1$ to $C_2$ proportional to $k_3$), (3) flow in from another compartment (like from $C_2$ to $C_1$ proportional to $k_4$), or (4) output that leaves the system (like that marked with constants $k_2$ and $k_5$). For a given compartmental model, each compartment will only have whichever of these makes sense for the model.  Thus, with this basic framework you can model different systems, even if both models are 2-compartment.

So far, we've been pretty generic.  We wanted to first get introduced to the basic concepts and terminology before getting our hands dirty.  But now we're ready for a real 2-compartment model that models the spread of diseases like the common cold that do not confer immunity after recovery.  This means people are either susceptible (S) to infection or are infectious (I) themselves.  Highly original scientists calls this the SI model of disease spread.  Let's look at the equations and diagram, then discuss it:

$
\begin{equation}
    \frac{dS(t)}{dt} = \mu N - \frac{\beta S I}{N} - \nu S  \\
    \frac{dI(t)}{dt} = \frac{\beta S I}{N} - \nu I
\end{equation}
$

![si_model](figs/si_model_cartoon.png "SI model")

$S(t)$ is the fraction of the population that's currently susceptible to infection, $I(t)$ is the fraction already infected, and $N=S+I$ is the total population.  The $\frac{dS(t)}{dt}$ equation is saying that 
* (term 1: $\mu N$) the $S$ population is increased over time by new births, with $\mu$ being the rate at which people are born for a population of the size $N$, 
* (term 2: $- \frac{\beta S I}{N}$) the $S$ population is decreased over time according to the rate ($\beta/N$) at which susceptible individuals ($S$) in contact with infectious individuals ($I$) become infected, leaving $S$ and joining $I$, and finally
* (term 3: $- \nu S$) the $S$ population is decreased over time acocrding to the mortality rate $\nu$.

Consider the $\frac{dI(t)}{dt}$ equation and see if you can express in words what each term represents, similar to what we've just done for $\frac{dS(t)}{dt}$.

At this point, you should be equipped with enough understanding of compartmental models to be able to carry out an inquiry into what ODE models like this can tell us and what constraints there may be when solving them computationally.  In summary, compartmental models are a framework for modeling systems that can be decomposed into interacting compartments through which some quantity (information, infection, nuclear matter) flows.

So, we have these things called ODEs (oridnary differential equations), which are at the heart of compartmental models (as well as many other modeling techniques).  What can we do with them?  In particular for this course, how can a computer be used to understand the properties and predictions of an ODE model?  And again, what can we do to build confidence that the predictions of a machine we've programmed are trustworthy?  In this inquiry activity, you'll end up investigating these questions.  But before getting there, we need to introduce one more essential computational tool you'll need to work with mathematical equations on a computer: **finite differences**, in particular we'll see one of the most fundamental such differences: the **forward Euler method**.

#  A basic finite difference: forward Euler

In the previous section, we explored a framework for specifying a set of mathematical equations (ODEs).  How do we go from our abstract mathematics to something a computer understands?  Doing so is at the core of the computational sciences, and you're about to learn (or perhaps refresh your memory of) one of the most ubiquitious and important tools for it: the finite difference.

Finite differences are one technique for approximating a derivative (in other words, the left-hand-side of an ODE).  In calculus and most math we may use day-to-day, mathematical functions tend to be continuous.  This means that even in the interval $x \in [0,1]$ there are an infinite number of possible values $x$ can take on when you calculate $f(x)$.  On a computer, however, we have a finite amount of memory available to represent numbersand in practice there's a limit to the smallest rational number you can represent.  A finite difference addresses this by breaking the numberline up into discrete chunks and approximates a derivative using data points from this discrete set of possible $x$ values.

The technique greatly predates the advent of computers, but is now widely used in research and production codes.  For the purposes of this activity, we'll just introduce one of the most basic finite difference methods: **Forward Euler**.  But before all that, let's step back a bit and consider the following continuous function $f(x)$ which has been overlayed on a numberline with some discrete points marked:

![findiff_numline](figs/findiff_numline.png "Finite difference number line")

In the above plot, the continuous variable $x$ is *discretized* into an equidistant series of values $x_0, x_1, x_2, ... , x_i , ...$.  We use the subscript index variable $i$ to represent a given value of $x$.  Things depending on $x$ will inherit the subscript, such as $f_i \equiv f(x_i)$.  

The continuous math is in red.  This is what a computer has trouble handling.  A few discrete points have been marked in blue: $f_i = f(x_i)$ and $f_{i+1} = f(x_{i+1})$.  In calculus, there's a technique we can use to represent _any_ function near a given value of $x$ in terms of the function's derivatives.  This technique is called Taylor expansion.  We won't go into the details, but using this technique we can find that

$
\begin{equation}
    f(x_{i+1}) = f(x_i) + \Delta x \frac{df}{dx} |_{x_i} + \mathcal{O}(\Delta x^2).
\end{equation}
$
Some of these terms are already labeled in the figure above.  The $|_{x_i}$ is a notation that means the derivative $\frac{df}{dx}$ is evaluated at $x=x_{i+1}$. The $\mathcal{O}(\Delta x^2)$ is a notation that means there are other terms in this equation, but they are all proporational to $\Delta x^2$ or higher powers of $\Delta x$.  For example $... + a \Delta x^2 + b \Delta x^3 + c \Delta x^4 + ...$.  

For the forward Euler method, we simply neglect all terms that are proportional to $\Delta x^2$ or higher.  That's why it is an approximation (and you may hear people say things like this method is _2nd-order accurate_, because errors are proportional to the second power of $\Delta x$).  This leaves us with

$
\begin{equation}
    f(x_{i+1}) \simeq f(x_i) + \Delta x \frac{df}{dx} |_{x_i}, \text{or:} \\  
    f_{i+1} \simeq f_i + \Delta x \frac{df}{dx} |_{x_i}
\end{equation}
$
Thus, we have an approximate equation for the _forward_ value $f_{i+1}$ in terms of $\Delta x$, $f_i$, and $\frac{df}{dx} |_{x_i}$.

The final equation, $f_{i+1} = f_i + \Delta x \frac{df}{dx} |_{x_i} $, is an example of a **finite difference equation**.  It allows us to calculate $f_{i+1}$ if we know $f_i$, the stepsize $\Delta x$, and if we have an expression describing $\frac{df}{dx}$.  Now, can you think of a time you've encountered expressions of the form $\frac{df}{dx}$?  What if I write it like $\frac{dC_j(t)}{dt}$?  Indeed, the ordinary differential equations we explored in the section on compartmental models give us expressions for things like $\frac{dC_j(t)}{dt}$.  We can use these expressions in a finite difference equation to calculate a computer approximation of as many values of $C_j$ as we want.

As we saw, a measure of the accuracy is the order of the terms neglected by our approximation. So if we choose a good step size, perhaps we can get good approximations of the functions in a compartmental model.  For our 1-compartment model, we had

$
\begin{equation}
    \frac{dC_1(t)}{dt} = K_1 u(t) - k_2 C_1(t),
\end{equation}
$

By using this in our finite difference equation, we get

$
\begin{equation}
    C_{1,i+1} = C_{1,i} + \Delta t \left( K_1 u_i - k_2 C_{1,i} \right).
\end{equation}
$

Thus, if we know some initial conditions ($C_{1,i}$, $u_i$), we can approximate the value of $C_1(t=t_i + \Delta t)$.  This value could then be plugged back into the equations to get the *next* value, and so on.  This process of successive approximations is sometimes called *integrating* an equation.  Recall that our analysis approximated the "forward" value of $f_{i+1}$ using a known $f_i$.  Thus, this difference equation is sometimes called a **forward difference**.  

For the purposes of this activity, we will not derive the other difference equations, but we will introduce them.  An analysis that approximates the "backward" value ($i-1$) can eventually yield the **backward difference** equation:

$
\begin{equation*}
    f_{i+1} = f_i + \Delta x \frac{df}{dx}|_{x_i}
\end{equation*}
$

An analysis that combines forward and backward difference equations can yield a **centered difference** equation:

$
\begin{equation*}
    f_{i+1} = f_{i-1} + 2 \Delta x \frac{df}{dx}|_{x_i}
\end{equation*}
$

Before ending this section, it's good to note that we can rearrange our equation to write a discrete approximation of a derivative, which can be useful if instead of having an ODE that gives you $\frac{df}{dx} |_{x_i}$ you have a function but want to calculate the derivative:

$
\begin{equation*}
    \frac{df}{dx} |_{x_i} \simeq \frac{\Delta f}{\Delta x} = \frac{f_{i+1} - f{i}}{x_{i+1} - x_i}
\end{equation*}
$

#  Putting it all together: Spreading Rumors

# Pre-Class Assignment